In [4]:
## imports 
import pandas as pd
import numpy as np
import yaml
import plotnine
from plotnine import *

## way to connect to mysql 

import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [2]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("../../private_data/qss20_w22_dbcred.yaml")


In [36]:
## connect to the database
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [53]:
q1 = """
select *,
case when INCIDENT_CITY = 'Chicago' then 'Yes'
else 'No'
end as in_chicago
from caseinit
"""

q1_res = pd.read_sql_query(q1, cnx)


q2 = """
select *,
case when INCIDENT_CITY = 'Chicago' then 'Yes'
else 'No'
end as in_chicago
from caseinit
where INCIDENT_CITY <> ""
"""
q2_res = pd.read_sql_query(q2, cnx)
#pd.crosstab(q1_res.in_chicago, q1_res.INCIDENT_CITY).T
#pd.crosstab(q2_res.in_chicago, q2_res.INCIDENT_CITY).T

q3 = """
select count(*) as count_cases,
is_in_diversion
from caseinit
where INCIDENT_CITY = "Chicago"
group by is_in_diversion
"""
q3_res = pd.read_sql_query(q3, cnx)
q3_res

q4 = """
select avg(divert_binary) as prop_divert
from (select *,
case when is_in_diversion = 'True' then 1
else 0
end as divert_binary
from caseinit) as d
where INCIDENT_CITY = "Chicago"
"""
q4_res = pd.read_sql_query(q4, cnx)
q4_res

,count_cases,is_in_diversion
0,1702,False
1,1394,True


,prop_divert
0,0.4503


In [54]:
# your code here
q1 = """
select *, 
CASE WHEN INCIDENT_CITY = 'Chicago' then 'yes' 
ELSE 'no' end as in_chicago
from caseinit
"""
res  = pd.read_sql_query(q1, cnx)
#res

#pd.crosstab(res.INCIDENT_CITY, res.in_chicago)

q1_noblank = """
select *, 
CASE WHEN INCIDENT_CITY = 'Chicago' then 'yes' 
ELSE 'no' end as in_chicago
from caseinit
where INCIDENT_CITY <> ""
"""


res_nob  = pd.read_sql_query(q1_noblank, cnx)
#pd.crosstab(res_nob.INCIDENT_CITY, res_nob.in_chicago)

q3 = """
select count(*) as count_divert, is_in_diversion
from caseinit
where INCIDENT_CITY = 'Chicago'
group by is_in_diversion
"""

res_div  = pd.read_sql_query(q3, cnx)
res_div

q4 = """
select avg(diversion_bool) as prop_div
from
(select *, 
CASE WHEN is_in_diversion = 'True' then 1
ELSE 0 
END as diversion_bool
from caseinit) as d 
where INCIDENT_CITY = 'Chicago'
"""

res_div_p  = pd.read_sql_query(q4, cnx)
res_div_p.head()


## step 5
query_propdiv_compare = """
select avg(is_div_bool) as prop_div, in_chicago
from  (select *,
CASE when is_in_diversion = 'True' then 1
ELSE 0
END as is_div_bool,
CASE when INCIDENT_CITY = 'Chicago' then 'Yes'
ELSE 'No'
END as in_chicago
from caseinit) as d
where INCIDENT_CITY <> ''
group by in_chicago
"""
prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)
prop_div_compare

,count_divert,is_in_diversion
0,1702,False
1,1394,True


,prop_div
0,0.4503


,prop_div,in_chicago
0,0.5956,No
1,0.4503,Yes


# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

3. Built on the query from step 2 (and/or modify to just focus on drug school and drug court) to find the (1) rate of Black defendants sent to drug court, (2) rate of white defendants sent to drug court, (3) rate of Black defendants sent to drug school, and (4) rate of white defendants sent to drug school


In [ ]:
# your code here